In [1]:
import pandas  as pd
import numpy   as np
import matplotlib.pyplot as plt
import pickle
import shap
from xgboost                 import XGBRegressor
from sklearn.model_selection import GridSearchCV, train_test_split # Maybe RandomizedSearchCV is more interesting
from sklearn.preprocessing   import OneHotEncoder

In [2]:
data_th = pd.read_csv('data/ECV_Th_2021.csv', delimiter='\t')
data_th = data_th.replace(' ', '')
data_th = data_th.replace('  ', '')
data_th = data_th.replace('', np.nan)
for column in range(data_th.shape[1]):
    if column == 1:
        continue
        
    if data_th.iloc[:,column].dtype in (int, object):
        data_th.iloc[:,column] = data_th.iloc[:,column].astype(float)


problematic_columns = (83,87,89,91,93,95,97,101,109,115,172,174,176,178)

for column in problematic_columns:
     data_th.iloc[:,column] = data_th.iloc[:,column].astype(float)

data_tp = pd.read_csv('data/ECV_Tp_2021.csv', delimiter='\t')
data_tp = data_tp.replace(' ', '')
data_tp = data_tp.replace('  ', '')
data_tp = data_tp.replace('', np.nan)

data_tp['PB030'] = data_tp['PB030'] // 100 # converting all members of the same house to the same identifier
data_tp = data_tp.groupby('PB030', as_index=False).mean() # grouping by house using the mean value

data_td = pd.read_csv('data/ECV_Td_2021.csv', delimiter='\t')
data_td = data_td.replace(' ', '')
data_td = data_td.replace('  ', '')
data_td = data_td.replace('', np.nan)

data_tr = pd.read_csv('data/ECV_Tr_2021.csv', delimiter='\t')
data_tr = data_tr.replace(' ', '')
data_tr = data_tr.replace('  ', '')
data_tr = data_tr.replace('', np.nan)

problematic_columns = (13,25)

for column in problematic_columns:
     data_tr.iloc[:,column] = data_tr.iloc[:,column].astype(float)
        
data_tr['RB030'] = data_tr['RB030'] // 100
data_tr = data_tr.groupby('RB030', as_index=False).mean()

data = data_th.merge(data_tp, left_on='HB030', right_on='PB030')
data = data.merge(data_td, left_on='HB030', right_on='DB030')
data = data.merge(data_tr, left_on='HB030', right_on='RB030')

ohe = OneHotEncoder()
transformed = ohe.fit_transform(data[['DB040']])
data[ohe.categories_[0]] = transformed.toarray()
data.drop(['DB040'], axis = 1, inplace= True)

Columns (83,87,89,91,93,95,97,101,109,115,172,174,176,178) have mixed types.Specify dtype option on import or set low_memory=False.
Columns (13,25) have mixed types.Specify dtype option on import or set low_memory=False.


In [3]:
data.to_csv('data/merged_data.csv', index=False)